# Categorical Encoders

All machine learning models are mathematical models that need numbers to work with. Categorical encoders are parts of the SciKit Learn library in Python to convert text data into numbers helping our predictive models to understand better.


### Ordinal Variable:
Ordinal variables show information about the order of choices. 

Example: Educational Level- elementary school, high school, graduate, post graduate rankd from 1 to 4.
         
         Weekdays-  Monday=1 and Sunday=7
         

### Nominal Variable:  
Nominal variable do not have order or ranking. 

Example: Colours- red, orange, green...
         
         Gender, Race...

Here, for gender if we assign Male=0, and Female=1, it would not interpret that Female is greater or superior than Male



## Label Encoder
Label Encoder encode labels between values 0 to number of distinct labels.

In [61]:
import pandas as pd

In [62]:
data=pd.read_csv("Data.csv")
data

,Country,Age,Salary,Purchased
0,France,44.0,72000.0,No
1,Spain,27.0,48000.0,Yes
2,Germany,30.0,54000.0,No
3,Spain,38.0,61000.0,No
4,Germany,40.0,NaN,Yes
5,France,35.0,58000.0,Yes
6,Spain,NaN,52000.0,No
7,France,48.0,79000.0,Yes
8,Germany,50.0,83000.0,No
9,France,37.0,67000.0,Yes


In [63]:
# replacing NaN values with the median of Age values and Salary values resp.
data.Age.fillna(data.Age.median(), inplace=True)
data.Salary.fillna(data.Salary.median(), inplace=True)
data

,Country,Age,Salary,Purchased
0,France,44.0,72000.0,No
1,Spain,27.0,48000.0,Yes
2,Germany,30.0,54000.0,No
3,Spain,38.0,61000.0,No
4,Germany,40.0,61000.0,Yes
5,France,35.0,58000.0,Yes
6,Spain,38.0,52000.0,No
7,France,48.0,79000.0,Yes
8,Germany,50.0,83000.0,No
9,France,37.0,67000.0,Yes


In [64]:
import sklearn
from sklearn.preprocessing import LabelEncoder   # importing LabelEncoder class from the sklearn library
labelencoder=LabelEncoder()              # creating class object labelencoder

data["Country"]=labelencoder.fit_transform(data["Country"])
data

# Here,  0 gets assigned to France  # [alphabetically]
#        1 gets assigned to Germany
#        2 gets assigned to Spain       

,Country,Age,Salary,Purchased
0,0,44.0,72000.0,No
1,2,27.0,48000.0,Yes
2,1,30.0,54000.0,No
3,2,38.0,61000.0,No
4,1,40.0,61000.0,Yes
5,0,35.0,58000.0,Yes
6,2,38.0,52000.0,No
7,0,48.0,79000.0,Yes
8,1,50.0,83000.0,No
9,0,37.0,67000.0,Yes


* Here, the numerical labels assigned to categorical variables are in alphabetical order.
* Hence, a learning algorithm would assume that "Spain" is greater or superior than "France" and "Germany" which is not in this case.
* So, for nominal data, using label encoders may not be such a good idea, instead one must use OneHotEncoding.

## Ordinal Encoder

Ordinal Encoder encode labels in an ordered feature. It is used when the labels represent some kind of ranking.

In [65]:
# creating data frame, named student consisting of Ordinal type categorical variables in edu_level column.

student= pd.DataFrame([["M", "medium"], ["M", "high"], ["F", "high"], ["F", "low"], ["M", "medium"]], columns=["gender", "edu_level"])
student

,gender,edu_level
0,M,medium
1,M,high
2,F,high
3,F,low
4,M,medium


In [66]:
# ordinal encoding edu_level
import category_encoders as ce
data_oe=ce.OrdinalEncoder(cols=["edu_level"])
data_oe.fit_transform(student)


# Here, encoding results medium=1, high=2, low=3 as ordinal encoder encodes in an ordered manner. But, as we can see that
# edu_level contains Ordinal Variable i.e high > medium > low. So, encoding should have been according to their ordinalty.

,gender,edu_level
0,M,1
1,M,2
2,F,2
3,F,3
4,M,1


In [67]:
# one way to achieve this is by using standard pandas Categorical constructor where ordinality can be predefined.
category= pd.Categorical(student["edu_level"], categories=["low", "medium", "high"], ordered=True)
category

[medium, high, high, low, medium]
Categories (3, object): [low < medium < high]

In [68]:
# pd.factorize() encodes categorical variable.
labels, unique= pd.factorize(category, sort=True)
student["edu_level"]= labels
student

# This gives high(=2) > medium(=1) > low(=0)

,gender,edu_level
0,M,1
1,M,2
2,F,2
3,F,0
4,M,1


In [69]:
# Ordinal encoding can also be done my map function
# creating dictionary to rank the edu_level
dictionary= {"low": 1, "medium": 2, "high": 3}
dictionary

{'low': 1, 'medium': 2, 'high': 3}

In [70]:
student["edu_level"]= student["edu_level"].map(dictionary)
student

,gender,edu_level
0,M,NaN
1,M,NaN
2,F,NaN
3,F,NaN
4,M,NaN


### Count Encoder or Frequency Encoder
Here the categorical labels are encoded to the number of times they occur

In [71]:
# titanic dataset
data= pd.read_csv("train.csv")
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Let's encode Embarked column

In [72]:
# since Embarked have only 2 null values, we can drop those rows
data= data.dropna(subset=["Embarked"])

In [73]:
# getting the frequency of labels in Embarked
data["Embarked"].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [74]:
# converting it to dictionary
mapped_value= (data["Embarked"].value_counts()).to_dict()
mapped_value

{'S': 644, 'C': 168, 'Q': 77}

In [75]:
#encoding using map function
data["Embarked"]= data["Embarked"].map(mapped_value)
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,644
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,168
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,644
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,644
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,644
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,644
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,644
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,644
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,168


One of the disadvantage of Frequency Encoder is that it will give same weights to the labels with the same frequency

## One Hot Encoder

Here, for rows having value as "monroe township", the town_1 column will have value "1" and the other two columns will have "0"s.

Similarly, for rows having value as "west windsor", the town_2 column will have value "1" and the other two columns will have "0"s.


In [102]:
home= pd.read_csv("homeprices.csv")
home.head()

,town,area,price
0,monroe township,2600,550000
1,monroe township,3000,565000
2,monroe township,3200,610000
3,monroe township,3600,680000
4,monroe township,4000,725000


In [77]:
data_onehot= ce.OneHotEncoder(cols=["town"])
data_onehot.fit_transform(home)

,town_1,town_2,town_3,area,price
0,1,0,0,2600,550000
1,1,0,0,3000,565000
2,1,0,0,3200,610000
3,1,0,0,3600,680000
4,1,0,0,4000,725000
5,0,1,0,2600,585000
6,0,1,0,2800,615000
7,0,1,0,3300,650000
8,0,1,0,3600,710000
9,0,0,1,2600,575000


## Binary Encoder

In this encoding, first the categorical variables are encoded as ordinal encoding, then those codes(integer value) are converted into binary codes, then the digits from that binary string are split into separate columns.

In [101]:
home= pd.read_csv("homeprices.csv")
home.head()

,town,area,price
0,monroe township,2600,550000
1,monroe township,3000,565000
2,monroe township,3200,610000
3,monroe township,3600,680000
4,monroe township,4000,725000


In [79]:
data_bin=ce.BinaryEncoder(cols=["town"]) # works on ordinal encoded data
data_bin.fit_transform(home)

,town_0,town_1,town_2,area,price
0,0,0,1,2600,550000
1,0,0,1,3000,565000
2,0,0,1,3200,610000
3,0,0,1,3600,680000
4,0,0,1,4000,725000
5,0,1,0,2600,585000
6,0,1,0,2800,615000
7,0,1,0,3300,650000
8,0,1,0,3600,710000
9,0,1,1,2600,575000


## BaseN Encoder    
BaseN Encoder encodes category into their base-N representation.
* base=1 is equivalent to one hot encoder
* base=2 is equivalent to binary encoder
* by default it takes base=2


In [103]:
data=pd.read_csv("Data.csv")
data.head()

,Country,Age,Salary,Purchased
0,France,44.0,72000.0,No
1,Spain,27.0,48000.0,Yes
2,Germany,30.0,54000.0,No
3,Spain,38.0,61000.0,No
4,Germany,40.0,NaN,Yes


In [81]:
# base=1, i.e encodes like OneHotEncoder
data_base=ce.BaseNEncoder(cols=["Country"], base=1)     
data_base.fit_transform(data)

,Country_0,Country_1,Country_2,Country_3,Age,Salary,Purchased
0,0,1,0,0,44.0,72000.0,No
1,0,0,1,0,27.0,48000.0,Yes
2,0,0,0,1,30.0,54000.0,No
3,0,0,1,0,38.0,61000.0,No
4,0,0,0,1,40.0,NaN,Yes
5,0,1,0,0,35.0,58000.0,Yes
6,0,0,1,0,NaN,52000.0,No
7,0,1,0,0,48.0,79000.0,Yes
8,0,0,0,1,50.0,83000.0,No
9,0,1,0,0,37.0,67000.0,Yes


In [82]:
# by default, takes base=2, i.e encodes like BinaryEncoder
data_base=ce.BaseNEncoder(cols=["Country"])    
data_base.fit_transform(data)

,Country_0,Country_1,Country_2,Age,Salary,Purchased
0,0,0,1,44.0,72000.0,No
1,0,1,0,27.0,48000.0,Yes
2,0,1,1,30.0,54000.0,No
3,0,1,0,38.0,61000.0,No
4,0,1,1,40.0,NaN,Yes
5,0,0,1,35.0,58000.0,Yes
6,0,1,0,NaN,52000.0,No
7,0,0,1,48.0,79000.0,Yes
8,0,1,1,50.0,83000.0,No
9,0,0,1,37.0,67000.0,Yes


# Hot One Encoding v/s Dummy Encoding

### Dummy Variable Trap

Dummy variable trap occurs when independent categorical variables are multicolinear.

Muticolinearity is the scenario where we can derive one variable from other variables.

For example: In homeprice.csv data, if monroe township is (1,0) and robinsville is (0,1) then west windsor will be, monroe township=0 and robinsville=0 i.e (0,0)


 

## Dummy Variable (indicator variable)

Dummy coded variables have values of 0 for the reference group and 1 for the comparison group.


In [104]:
home= pd.read_csv("homeprices.csv")
home.head()

,town,area,price
0,monroe township,2600,550000
1,monroe township,3000,565000
2,monroe township,3200,610000
3,monroe township,3600,680000
4,monroe township,4000,725000


In [84]:
# pandas has a function pd.get_dummies() to create dummy variable

dummy= pd.get_dummies(home["town"])
dummy

# for rows having value as "monroe township", the monroe township column will have value "1" and the other two columns will 
# have "0"s.
# Similarly, for rows having value as "west windsor", the town_2 column will have value "1" and the other two columns will 
# have "0"s.

,monroe township,robinsville,west windsor
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
5,0,0,1
6,0,0,1
7,0,0,1
8,0,0,1
9,0,1,0


In [85]:
# Dropping west windsor column as it is clear that, when monroe township=(1 0) and robinsville=(0 1), west windsor will be (0 0)


dummy.drop("west windsor", axis="columns", inplace=True)
dummy

,monroe township,robinsville
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,1


## Sum Encoder

In [105]:
home= pd.read_csv("homeprices.csv")
home.head()

,town,area,price
0,monroe township,2600,550000
1,monroe township,3000,565000
2,monroe township,3200,610000
3,monroe township,3600,680000
4,monroe township,4000,725000


In [87]:
data_sum= ce.SumEncoder(cols=["town"])
data_sum.fit_transform(home)

# last independent variable(robinsville) is taken as reference giving it a value of -1. 
# town_0 and town_1 represents monroe township and west windsor resp.
# The independent variable being compared(monroe township and west windsor) is given a value of 1 in their resp. columns.

,intercept,town_0,town_1,area,price
0,1,1.0,0.0,2600,550000
1,1,1.0,0.0,3000,565000
2,1,1.0,0.0,3200,610000
3,1,1.0,0.0,3600,680000
4,1,1.0,0.0,4000,725000
5,1,0.0,1.0,2600,585000
6,1,0.0,1.0,2800,615000
7,1,0.0,1.0,3300,650000
8,1,0.0,1.0,3600,710000
9,1,-1.0,-1.0,2600,575000


### Target Guided Ordinal Encoding
Categorical labels are ordered according to the target variable.

In [88]:
home= pd.read_csv("homeprices.csv")
home.head()

,town,area,price
0,monroe township,2600,550000
1,monroe township,3000,565000
2,monroe township,3200,610000
3,monroe township,3600,680000
4,monroe township,4000,725000


In [89]:
# the labels in town are grouped according to the price(target variable)
home.groupby(["town"])["price"].mean()

town
monroe township    626000
robinsville        622500
west windsor       640000
Name: price, dtype: int64

In [90]:
# ordering town in ascending order
home.groupby(["town"])["price"].mean().sort_values()

town
robinsville        622500
monroe township    626000
west windsor       640000
Name: price, dtype: int64

In [91]:
# gettint the index
labels= home.groupby(["town"])["price"].mean().sort_values().index
labels

Index(['robinsville', 'monroe township', 'west windsor'], dtype='object', name='town')

In [92]:
# ordering the labels and converting it to dictionary
ordered_labels= {k:i for i,k in enumerate(labels,0)}
ordered_labels

{'robinsville': 0, 'monroe township': 1, 'west windsor': 2}

In [93]:
# encoding by map function
home["town_encoded"]= home["town"].map(ordered_labels)
home

,town,area,price,town_encoded
0,monroe township,2600,550000,1
1,monroe township,3000,565000,1
2,monroe township,3200,610000,1
3,monroe township,3600,680000,1
4,monroe township,4000,725000,1
5,west windsor,2600,585000,2
6,west windsor,2800,615000,2
7,west windsor,3300,650000,2
8,west windsor,3600,710000,2
9,robinsville,2600,575000,0


### Mean Encoder
Categorical labels are encoded by the mean of the target variable.

In [94]:
home= pd.read_csv("homeprices.csv")
home.head()

,town,area,price
0,monroe township,2600,550000
1,monroe township,3000,565000
2,monroe township,3200,610000
3,monroe township,3600,680000
4,monroe township,4000,725000


In [95]:
# the labels in town are grouped according to the mean of price(target variable)
home.groupby(["town"])["price"].mean()

town
monroe township    626000
robinsville        622500
west windsor       640000
Name: price, dtype: int64

In [96]:
# converting it to dictionary
encoded_labels= home.groupby(["town"])["price"].mean().to_dict()
encoded_labels

{'monroe township': 626000, 'robinsville': 622500, 'west windsor': 640000}

In [97]:
# encoding by map function
home["town_mean_encoded"]= home["town"].map(encoded_labels)
home

,town,area,price,town_mean_encoded
0,monroe township,2600,550000,626000
1,monroe township,3000,565000,626000
2,monroe township,3200,610000,626000
3,monroe township,3600,680000,626000
4,monroe township,4000,725000,626000
5,west windsor,2600,585000,640000
6,west windsor,2800,615000,640000
7,west windsor,3300,650000,640000
8,west windsor,3600,710000,640000
9,robinsville,2600,575000,622500


In [98]:
# mean encoding by categorical encoder library
home_targ= ce.TargetEncoder(cols=["town"])
home_targ.fit(home["town"], home["price"])
home_targ.transform(home["town"], home["price"])

,town
0,626058.109294
1,626058.109294
2,626058.109294
3,626058.109294
4,626058.109294
5,639489.259827
6,639489.259827
7,639489.259827
8,639489.259827
9,622819.212608


## Leave One Out Encoder

Each categorical label is encoded by the mean value of their respective outcome excluding current label.

In [106]:
home= pd.read_csv("homeprices.csv")
home.head()

,town,area,price
0,monroe township,2600,550000
1,monroe township,3000,565000
2,monroe township,3200,610000
3,monroe township,3600,680000
4,monroe township,4000,725000


In [100]:
home_leave= ce.LeaveOneOutEncoder(cols=["town"])
home_leave.fit(home["town"], home["price"])
home_leave.transform(home["town"], home["price"])

# label1 encoded as- mean of "price"(outcome) values of monroe township excluding 1st one= (565000 + 610000 + 680000 + 725000 + )/4
# label2 encoded as- mean of "price"(outcome) values of monroe township excluding 2nd one= (550000 + 610000 + 680000 + 725000 + )/4
# label3 encoded as- mean of "price"(outcome) values of monroe township excluding 3rd one= (550000 + 565000 + 680000 + 725000 + )/4
# and so on...

,town
0,645000.000000
1,641250.000000
2,630000.000000
3,612500.000000
4,601250.000000
5,658333.333333
6,648333.333333
7,636666.666667
8,616666.666667
9,638333.333333
